In [117]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm



import warnings

warnings.filterwarnings('ignore')

In [119]:
df_MAD = pd.read_parquet(f'./dbs/intermediate/indi_weights.parquet')


In [120]:
df_survey_twins = pd.read_pickle(f'./dbs/intermediate/df_survey_twins.pkl')


# add region information

In [121]:
lan= pd.read_csv(f'./dbs/lan.csv', delimiter=';', dtype={'code':'string'})
lan.rename(columns={'code':'county', 'landsdelar':'region'},inplace=True)

df_MAD = pd.merge(df_MAD, lan[['county','region']], on='county')
df_survey_twins = pd.merge(df_survey_twins, lan[['county','region']], on='county')


# calculate median distance

In [122]:
avg_dist_threshold = df_survey_twins.avg_dist.median()
com_dist_threshold = df_survey_twins.com_distance.median()


# create stratum

In [123]:
df_MAD['region'] = df_MAD['region'].replace({'Svealand':'0','Göteland':'1', 'Norrland':'2'})

df_MAD['urban_density'] = df_MAD['urban_density'].replace({'A': '0', 'B': '1', 'C': '1'})

df_MAD['avg_dist_cat'] = '0'
df_MAD['avg_dist_cat'][df_MAD.avg_dist > avg_dist_threshold] = '1'

# df_MAD['com_dist_cat'] = '0'
# df_MAD['com_dist_cat'][df_MAD.com_dist > com_dist_threshold] = '1'

df_MAD.rename(columns={'commute':'employee'}, inplace=True)

In [124]:

df_MAD['stratum'] = df_MAD['region'].astype(str) + df_MAD['urban_density'].astype(str) +  df_MAD['employee'].astype(str)+  df_MAD['avg_dist_cat'].astype(str)

# df_MAD['stratum'][(df_MAD.commute == 1)] = df_MAD['stratum'][(df_MAD.commute == 1)] + df_MAD['com_dist_cat'][(df_MAD.commute == 1)].astype(str)

df_MAD['stratum'].value_counts()

0101    48915
0100    37723
1101    34118
1100    33219
0111    13312
1110    13226
1111    12411
2100    11705
0110    11580
2101     7158
1001     6322
0001     4944
2110     4158
1011     3365
1000     3215
2111     2502
2001     2399
0011     2054
0000     1899
2000     1344
2011     1136
1010     1055
0010      561
2010      346
Name: stratum, dtype: int64

In [125]:
df_survey_twins['region'] = df_survey_twins['region'].replace({'Svealand':'0','Göteland':'1', 'Norrland':'2'})

df_survey_twins['urban_density'] = df_survey_twins['urban_density'].replace({'A': '0', 'B': '1', 'C': '1'})

df_survey_twins['avg_dist_cat'] = '0'
df_survey_twins['avg_dist_cat'][df_survey_twins.avg_dist > avg_dist_threshold] = '1'

# df_survey_twins['com_dist_cat'] = '0'
# df_survey_twins['com_dist_cat'][df_survey_twins.com_distance > com_dist_threshold] = '1'


In [126]:

df_survey_twins['stratum'] = df_survey_twins['region'].astype(str) + df_survey_twins['urban_density'].astype(str) +  df_survey_twins['employee'].astype(str)+  df_survey_twins['avg_dist_cat'].astype(str)

# df_survey_twins['stratum'][(df_survey_twins.commute == 1)] = df_survey_twins['stratum'][(df_survey_twins.commute == 1)] + df_survey_twins['com_dist_cat'][(df_survey_twins.commute == 1)].astype(str)

df_survey_twins['stratum'].value_counts()

1111    3006
1110    2953
1100    1719
0110    1309
0111    1256
1101     869
1011     868
0100     724
1001     339
0101     330
2110     316
0011     243
2100     168
2111     153
1010     145
2011     132
0001     103
1000      78
2001      78
2101      61
0010      16
0000       8
2010       3
Name: stratum, dtype: int64

In [127]:
list_survey_type = list(df_survey_twins['stratum'].unique())
list_MAD_type = list(df_MAD['stratum'].unique())


#set(list_MAD_type).difference(set(list_survey_type))

In [128]:
threshold_count = 50

df_stratum = pd.DataFrame(columns=['stratum','level','mad_size','sample_size' , 'sur_size','survey_ppl'])


for i in list_MAD_type:
    df_MAD_p_type = df_MAD[df_MAD['stratum']==i]
    #print('mad len: ', len(df_MAD_p_type))

    df_sur_p_type = df_survey_twins[df_survey_twins['region']==i[0]]

    df_sur_temp = df_sur_p_type
    level = 0
    if len(df_sur_p_type) > threshold_count:
        df_sur_temp = df_sur_p_type[df_sur_p_type['urban_density']==i[1]]
        level = 1
        if len(df_sur_temp) > threshold_count:
            df_sur_p_type = df_sur_temp
            df_sur_temp = df_sur_p_type[df_sur_p_type['employee']==int(i[2])]
            level = 2
            #print(level)
            if len(df_sur_temp) > threshold_count:
                df_sur_p_type = df_sur_temp
                df_sur_temp = df_sur_p_type[df_sur_p_type['avg_dist_cat']==i[3]]
                level = 3
                #print(int(i[2]))
                if len(df_sur_temp) > threshold_count:
                   df_sur_p_type = df_sur_temp
                   level = 4

                # elif (len(df_sur_temp) > threshold_count)&(int(i[2])==1):
                #     df_sur_p_type = df_sur_temp
                #     df_sur_temp = df_sur_p_type[df_sur_p_type['com_dist_cat']==i[4]]
                #     level = 4
                #     #print(level)
                #
                #     if len(df_sur_temp) > threshold_count:
                #         df_sur_p_type = df_sur_temp
                #         level = 5

    df_stratum = df_stratum.append(dict(stratum=i, level=level, mad_size=df_MAD_p_type.wt.sum(), sample_size=len(df_MAD_p_type), sur_size=len(df_sur_p_type), survey_ppl=df_sur_p_type['sub_id'].tolist()), ignore_index=True)



## simplify activity sequences

In [129]:

# def act_seq_simplifier(data):
#     alist = list(data['act_seq'])
#     counter = 0
#     for i in range(len(alist) - 1, 0, -1):
#         if alist[i] in ['Home', 'Work', 'School']:
#             counter = 0
#             if alist[i] == alist[i-1]:
#                 del alist[i]
#         elif alist[i] == 'Other':
#             counter += 1
#             if counter > 3:
#                 del alist[i]
#     data['act_uniq'] = tuple(alist)
#     return data

# tqdm.pandas()
# df_survey_twins = df_survey_twins.apply(lambda row: act_seq_simplifier(row), axis=1)

In [130]:
seq_dist = df_survey_twins.act_seq_simp.value_counts(normalize=True).reset_index(name='count_desired')
seq_dist['count_desired']=seq_dist['count_desired']*df_stratum.mad_size.sum()

## generate candidates df by stratums

In [131]:
df_candidates = pd.DataFrame(columns=['sub_id', 'act_seq', 'act_seq_simp','stratum','mad_size' ,'ratio'])


for index, row in df_stratum.iterrows():

    df_temp= df_survey_twins[['sub_id', 'act_seq', 'act_seq_simp']][df_survey_twins.sub_id.isin(row['survey_ppl'])]
    df_temp['stratum']= row['stratum']
    df_temp['mad_size']=row['mad_size']
    df_temp['ratio']=row['mad_size']/row['sur_size']
    df_candidates = df_candidates.append(df_temp)
    #print(row['survey_ppl'])
    #len(df_survey_twins[df_survey_twins.sub_id.isin(row['survey_ppl'])])



In [132]:
df_candidates.ratio.sum()

7438637.716134105

In [133]:
#df_candidates.groupby('act_uniq')['ratio'].sum()

## fitting probabilities to the simplified activity seq distribution

In [134]:
for i in range(50):

    #fit to candidates ratios to the act seq distribution
    bbb = df_candidates.groupby('act_seq_simp')['ratio'].sum().reset_index(name='count')
    merged_df = pd.merge(seq_dist, bbb, left_on='index', right_on='act_seq_simp', how='inner')
    merged_df['result_column'] = merged_df['count_desired'] / merged_df['count']
    df_candidates = pd.merge(df_candidates, merged_df[['act_seq_simp', 'result_column']], on='act_seq_simp')

    df_candidates['ratio']= df_candidates['ratio']*df_candidates['result_column']
    df_candidates.drop(columns='result_column', inplace=True)

    #here tone down very high probabilities
    # w0 = ((np.std(df_candidates.loc[:, 'ratio']) / np.mean(df_candidates.loc[:, 'ratio'])) ** 2 + 1) ** 0.5 * 3.5 * np.median( df_candidates.loc[:, 'ratio'])
    # df_candidates.loc[df_candidates['ratio'] > w0, 'ratio'] = w0

    #fit to candidates ratios to the mad size by stratum
    bbb = df_candidates.groupby('stratum')['ratio'].sum().reset_index(name='count')
    df_candidates = pd.merge(df_candidates, bbb, on='stratum')

    df_candidates['ratio'] = df_candidates['ratio'] * (df_candidates['mad_size'] / df_candidates['count'])
    df_candidates.drop(columns='count', inplace=True)





## sampling survey ppl by the calculated probabilities for each stratum

In [135]:
df_candidates['probability'] = df_candidates['ratio']/df_candidates['mad_size']

#df_MAD['twin_ppl'] =0

In [136]:
df_candidates_prob= df_candidates.groupby('stratum')['probability'].apply(list).reset_index(name='probability_list')


df_candidates_ppl= df_candidates.groupby('stratum')['sub_id'].apply(list).reset_index(name='ppl_list')

In [137]:
df_stratum = pd.merge(df_stratum, df_candidates_prob)

df_stratum = pd.merge(df_stratum, df_candidates_ppl)


In [138]:
df_stratum['twin_ppl'] = df_stratum.apply( lambda row: np.argmax(np.random.multinomial(1, row['probability_list'], row['sample_size']), axis=1).tolist()  , axis=1)

## matching sampled survey ppl with MAD ppl

In [139]:
df_survey_twins_new = df_survey_twins.set_index('sub_id')
df_survey_twins_new = df_survey_twins_new[['act_seq', 'act_seq_simp', 'avg_dist','commute']]

In [140]:
from collections import Counter


def other_count(data):
    act_tuple = data['act_seq_simp']
    other_count_seq = Counter(act_tuple).get('Other')
    if other_count_seq is not None:
        data['other_count_seq'] = other_count_seq
    else:
        data['other_count_seq'] = 0
    return data

In [141]:

tqdm.pandas()
df_survey_twins_new = df_survey_twins_new.apply(lambda row: other_count(row), axis=1)


In [142]:
# df_twins = pd.DataFrame()
#
# for index, row in df_stratum.iterrows():
#     twin_ppl_list = df_stratum.loc[index,'ppl_list']
#     twin_ppl_place = df_stratum.loc[index,'twin_ppl']
#     twin_ppl_list = [twin_ppl_list[place] for place in twin_ppl_place]
#
#     df_survey_candidates= df_survey_twins_new.loc[twin_ppl_list].reset_index()
#
#     df_survey_candidates.sort_values(by=['other_count_seq','avg_dist'], inplace=True)
#     df_survey_candidates.reset_index(inplace=True)
#     df_survey_candidates.drop(columns='index', inplace=True)
#
#     df_MAD_new = df_MAD[df_MAD.stratum==row['stratum']]
#     df_MAD_new.sort_values(by=['other_count','avg_dist'], inplace=True)
#     df_MAD_new.reset_index(inplace=True)
#     df_MAD_new.drop(columns='index', inplace=True)
# #
#     df_MAD_new = pd.concat([df_MAD_new, df_survey_candidates], axis=1, ignore_index=False)
#     df_twins = df_twins.append(df_MAD_new)




In [143]:
df_twins = pd.DataFrame()

for index, row in df_stratum.iterrows():
    twin_ppl_list = df_stratum.loc[index,'ppl_list']
    twin_ppl_place = df_stratum.loc[index,'twin_ppl']
    twin_ppl_list = [twin_ppl_list[place] for place in twin_ppl_place]

    df_survey_candidates= df_survey_twins_new.loc[twin_ppl_list].reset_index()

    df_MAD_new = df_MAD[df_MAD.stratum==row['stratum']]

    zero_MAD = df_MAD_new[df_MAD_new.other_count == 0]
    zero_survey = df_survey_candidates[df_survey_candidates.other_count_seq == 0]

    #for i in range(len(df_MAD_new.other_count_seq.max())):
    if len(zero_MAD) <= len(zero_survey):

        zero_survey.sort_values(by=['avg_dist'], inplace=True)
        zero_survey.reset_index(inplace=True)
        zero_survey = zero_survey.iloc[:len(zero_MAD)]
        remain_survey = df_survey_candidates.drop(index=zero_survey.index.to_list())
        zero_survey.drop(columns='index', inplace=True)

        zero_MAD.sort_values(by=['avg_dist'], inplace=True)
        zero_MAD.reset_index(inplace=True)
        zero_MAD.drop(columns='index', inplace=True)

        zero_MAD = pd.concat([zero_MAD, zero_survey.iloc[:len(zero_MAD)]], axis=1, ignore_index=False)


    else:
        print(row['stratum'])

    df_twins = df_twins.append(zero_MAD)

    remain_MAD = df_MAD_new[df_MAD_new.other_count > 0]


    remain_survey.sort_values(by=['avg_dist'], inplace=True)
    remain_survey.reset_index(inplace=True)
    remain_survey.drop(columns='index', inplace=True)

    remain_MAD.sort_values(by=['avg_dist'], inplace=True)
    remain_MAD.reset_index(inplace=True)
    remain_MAD.drop(columns='index', inplace=True)

    remain_MAD = pd.concat([remain_MAD, remain_survey], axis=1, ignore_index=False)
    df_twins = df_twins.append(remain_MAD)

    #print(index)





In [144]:
filename = r'.\dbs\twins\run-evaluate_1.pkl'
# %store -r filename


df_twins.to_pickle(filename)

## Evaluation

In [145]:
print('The share of the matched survey ppl: ', np.round(len(df_twins.sub_id.unique())/len(df_survey_twins.sub_id.unique()),2))

The share of the matched survey ppl:  0.95


In [28]:
# print('Repetition number of 10 most matched people \n', df_twins.sub_id.value_counts().head(10))

Repetition number of 10 most matched people 
 20110931074    196
20142021076    195
20135211019    195
20111611003    194
20124851036    194
20111541082    190
20112511002    189
20142641064    189
20112221001    185
20112621014    184
Name: sub_id, dtype: int64


# code ends here

In [280]:
# len(df_twins[(df_twins.employee== 1) ])

65706

In [281]:
# len(df_twins[(df_twins.employee== 1)& (df_twins.commute== 0)])

4593

In [331]:
# df_twins['wt'][(df_twins.employee== 1) ].sum()/df_twins['wt'].sum()

0.5569376905013758

In [332]:
# df_twins['wt'][(df_twins.employee== 1)& (df_twins.commute== 1) ].sum()/df_twins['wt'][(df_twins.employee== 1) ].sum()

0.9281932113744805